In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
import gc
from tqdm import tqdm

%matplotlib inline

PATH = '/kaggle/input/idao2020-track1/'

In [2]:
!pip install /kaggle/input/lib-plyades

Processing /kaggle/input/lib-plyades
  Created wheel for Plyades: filename=Plyades-0.0.1-cp36-none-any.whl size=18914 sha256=39bd3d9e282c4fc886104c3a8b91afde14b38862d186e7b1b73eb9a93bae8224
  Stored in directory: /root/.cache/pip/wheels/91/67/f4/51685f7b31e174b1573e464db1cc864edffe5d36c233556335
Successfully built Plyades


In [3]:
!pip install jplephem

  Created wheel for jplephem: filename=jplephem-2.12-cp36-none-any.whl size=43929 sha256=3df44eb749536a21e84c8d24e6abc7b1c0de26e34d50fdaf71afa15eb88a9b06
  Stored in directory: /root/.cache/pip/wheels/08/ce/59/bde14a0c20cdfb538a28d78fec79dd628c8d03d5a75f812d1a
Successfully built jplephem


In [4]:
import plyades as pl
from astropy.time import Time
import astropy.units as units

In [5]:
%%time
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'test.csv')
submission = pd.read_csv(PATH + 'submission.csv')

CPU times: user 3.67 s, sys: 330 ms, total: 4 s
Wall time: 4.01 s


In [6]:
train.head()

,id,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,0,2014-01-01T00:00:00.000,0,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133
1,1,2014-01-01T00:46:43.000,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468
2,2,2014-01-01T01:33:26.001,0,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768
3,3,2014-01-01T02:20:09.001,0,-9148.251857,-20651.437460,-20720.381279,0.715600,-3.373762,1.722115,-9149.620794,-20618.200201,-20765.019094,0.712437,-3.375202,1.718306
4,4,2014-01-01T03:06:52.002,0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237


In [7]:
def make_iss(x, y, z, Vx, Vy, Vz, time):
    iss_r = np.array([x, y, z]) * units.km
    iss_v = np.array([Vx, Vy, Vz]) * units.km/units.s
    iss_t = Time(time)
    frame = 'ECI'
    body = pl.bodies.EARTH
    iss = pl.State(iss_r, iss_v, iss_t, frame, body)
    return iss

def generate_all_features(x, y, z, Vx, Vy, Vz, time):
    iss = make_iss(x, y, z, Vx, Vy, Vz, time)
    return (
        iss.semi_major_axis.value,
        iss.eccentricity.value,
        iss.inclination.value,
        iss.ascending_node.value,
        iss.argument_of_periapsis.value,
        iss.true_anomaly.value,
        iss.period.value,
        iss.orbital_energy.value,
        iss.mean_motion.value
    )

In [8]:
tqdm.pandas()
train[[
    'semi_major_axis',
    'eccentricity',
    'inclination',
    'ascending_node',
    'argument_of_periapsis',
    'true_anomaly',
    'period',
    'orbital_energy',
    'mean_motion'
]] = train.progress_apply(lambda s: pd.Series(generate_all_features(s['x_sim'], s['y_sim'], s['z_sim'], s['Vx_sim'], s['Vy_sim'], s['Vz_sim'], s['epoch'])), axis=1)

100%|██████████| 649912/649912 [5:40:56<00:00, 31.77it/s]


In [9]:
train.to_csv('train_plyades.csv', index=False)